***
## Получение изображения случайного котика

API-сервис thecatapi.com условно-бесплатен: при бесплатном доступе ограничено количество запросов и недоступны некоторые сервисы.

Получить случайную картинку можно бесплатно и без регистрации: нужно просто сделать GET-запрос к нужному эндпоинту. Перейдите по ссылке https://api.thecatapi.com/v1/images/search и посмотрите на ответ API:

![alt text](https://pictures.s3.yandex.net/resources/image_1709110682.png)

Ответ в формате JSON, в нём указан ID картинки, её URL и размер. Если обновить страницу, ответ изменится: при каждом запросе API возвращает новый адрес. 

Откройте URL картинки в браузере — и увидите саму картинку.

![alt text](https://pictures.s3.yandex.net/resources/image_1709110693.png)

Источник картинок найден! Когда-нибудь у KittyBot будет и своя база котиков, но для старта достаточно и этой. Но мало найти картинку: теперь её надо отправить пользователю.

***
## Отправка изображений через бота

Мессенджер Telegram умеет отправлять картинки; скорее всего, в библиотеке **pyTelegramBotAPI** должен быть метод для отправки изображений. Стас покопался в [документации](https://pytba.readthedocs.io/ru/latest/sync_version/index.html#telebot.TeleBot.send_photo) — да, так и есть: у объектов бота есть метод `send_photo()`.

Этот метод работает сходно с методом `send_message()`: в качестве параметров он принимает ID чата и ссылку на изображение. Однако получатель увидит именно изображение, а не ссылку на него.

Для начала стоит протестировать работу этого метода в простом коде: явно указать ID чата и точную ссылку — и отправить картинку из бота в свой персональный Telegram-аккаунт.

В каталоге проекта /kittybot создайте файл для экспериментов, например, send_random_image.py. 

Добавьте в файл код из листинга, укажите свой токен и `chat_id`; выполните код.


In [ ]:
# kittybot/send_random_image.py
from telebot import TeleBot


bot = TeleBot(token='<token>')

URL = 'https://cdn2.thecatapi.com/images/3dl.jpg'

chat_id = <chat_id>
text = 'Вам телеграмма!'
# Отправка сообщения
bot.send_message(chat_id, text)
# Отправка изображения
bot.send_photo(chat_id, URL) 

Если всё сделано верно, то вам в Telegram придёт фотография.

Осталось два вопроса:

* как сделать GET-запрос к API thecatapi.com;

* как из ответа API получать ссылку на изображение.

***
## Библиотека requests: запросы из кода

Продолжим эксперименты в файле *send_random_image.py*: импортируем в него библиотеку requests и немного обновим код.


In [5]:
# kittybot/send_random_image.py
import requests  # Импортируем библиотеку для работы с запросами.
from telebot import TeleBot


bot = TeleBot(token='8364619889:AAGCw6gZ24M6EIBySwfUDPpJ91EUxndTu7g')
# Адрес API сохраним в константе:
URL = 'https://api.thecatapi.com/v1/images/search'  
# Сделаем GET-запрос к API;
# метод json() преобразует полученный ответ JSON в тип данных, понятный Python.
response = requests.get(URL).json()

# Рассмотрим структуру и содержимое переменной response:
print(response)

# response — это список. Удостоверимся:
print(type(response))

# Какой длины список response?
print(len(response))

# Посмотрим, какого типа первый элемент:
print(type(response[0])) 

[{'id': 'bkm', 'url': 'https://cdn2.thecatapi.com/images/bkm.jpg', 'width': 650, 'height': 669}]
<class 'list'>
1
<class 'dict'>


В переменной `response` хранится ответ API в формате списка; единственный элемент этого списка — словарь с данными о картинке.


Ищем в словаре ссылку на изображение — и находим её под ключом `url`. Теперь извлечь её из ответа и передать в метод `send_photo()` — дело техники.


In [6]:
# kittybot/send_random_image.py
from telebot import TeleBot
import requests


bot = TeleBot(token='8364619889:AAGCw6gZ24M6EIBySwfUDPpJ91EUxndTu7g')
URL = 'https://api.thecatapi.com/v1/images/search'
chat_id = 1811456266

# Делаем GET-запрос к эндпоинту:
response = requests.get(URL).json()
# Извлекаем из ответа URL картинки:
random_cat_url = response[0].get('url')  

# Передаём chat_id и URL картинки в метод для отправки фото:
bot.send_photo(chat_id, random_cat_url) 

Готово: при каждом выполнении кода программа будет получать случайное изображение котика и отправлять его в чат с указанным ID. 

***
## Окончательный план работы

Новые знания — новые идеи. Немного изменим алгоритм работы KittyBot и приветственную фразу.

При получении команды /start бот должен:

* ответить персонализированным приветствием,

* отправить случайное изображение котика.

При получении команды /newcat бот должен отправить случайное изображение котика.

Добавим обработчик команды /newcat и обновим код всего бота.


In [ ]:
# kittybot/kittybot.py
import requests
from telebot import TeleBot


bot = TeleBot(token='<token>')
URL = 'https://api.thecatapi.com/v1/images/search'

# Код запроса к thecatapi.com и обработку ответа обернём в функцию:
def get_new_image():
    response = requests.get(URL).json()
    random_cat = response[0].get('url')
    return random_cat


# Добавляем хендлер для команды /newcat:
@bot.message_handler(commands=['newcat'])
def new_cat(message):
    chat = message.chat
    bot.send_photo(chat.id, get_new_image())


@bot.message_handler(commands=['start'])
def wake_up(message):
    chat = message.chat
    name = message.chat.first_name

    bot.send_message(
        chat_id=chat.id,
        text=f'Привет, {name}. Посмотри, какого котика я тебе нашёл',
    )

    bot.send_photo(chat.id, get_new_image())


@bot.message_handler(content_types=['text'])
def say_hi(message):
    chat = message.chat
    chat_id = chat.id
    bot.send_message(chat_id=chat_id, text='Привет, я KittyBot!')

bot.polling() 

После изменения кода перезапустите программу, а в чате Telegram очистите историю, чтобы бот вернулся в неактивное состояние и для него вновь стала доступна команда /start. 